In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patheffects as path_effects
import seaborn as sns
from sklearn.metrics import roc_curve, roc_auc_score, auc, precision_recall_curve
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from tqdm import tqdm
from functools import reduce

from src.config import *
from src.prediction_functions import *
from src.model_creation import *


# Functions

In [3]:
def set_perturabtion_type_values(perturbation, pert_type_lists):
    if perturbation in pert_type_lists['compound_list']: perturbation_type = 'cp'
    elif perturbation in pert_type_lists['ligand_list']: perturbation_type = 'lig'
    elif perturbation in pert_type_lists['receptor_list']: perturbation_type = 'rec'
    else: perturbation_type = np.nan
    return perturbation_type

In [4]:
def determine_type_of_perturbation_of_samples(data_index, data_index_perturbation, pert_type_lists):
    pert_types = pd.DataFrame(columns = ['perturbation', 'perturbation_type'], dtype='str')
    pert_types['perturbation'] = data_index_perturbation
    pert_types['perturbation_type'] = pert_types['perturbation'].apply(lambda x: set_perturabtion_type_values(x, pert_type_lists)) 
    pert_types.index = data_index
    return pert_types

In [5]:
def calculate_roc_auc(estimated_activities, true_matrix):
    # for every ligand calculate rocauc
    rocaucs = {}
    for ligand in estimated_activities.columns:
        if true_matrix.loc[:, ligand].sum() < 1:
            print(ligand, ' not considered')
            continue
        rocaucs[ligand] = roc_auc_score(y_true = true_matrix.loc[:, ligand], y_score = estimated_activities.loc[:, ligand])
    return rocaucs

In [6]:
def calculate_roc_auc_sample(estimated_activities, true_matrix):
    # for every ligand calculate rocauc
    rocaucs = {}
    for sample in estimated_activities.index:
        if true_matrix.loc[sample, :].sum() < 1:
            print(sample, ' not considered')
            continue
        rocaucs[sample] = roc_auc_score(y_true = true_matrix.loc[sample, :], y_score = estimated_activities.loc[sample, :])
    return rocaucs

In [7]:
def plot_rocauc_violin(data):
    fig, ax = plt.subplots(figsize = (2,5))
    sns.violinplot(data = list(data.values()), orient = 'v', palette = 'Pastel1')
    plt.xlabel('ROCAUC')
    plt.axhline(0.5, color = 'k')
    plt.show()

In [8]:
def plot_rocauc_boxplot(data):
    fig, ax = plt.subplots(figsize = (2,5))
    sns.boxplot(data = list(data.values()), orient = 'v', palette = 'Pastel1')
    sns.swarmplot(data = list(data.values()), color="gray", alpha = 0.7)

    plt.xlabel('ROCAUC')
    plt.axhline(0.5, color = 'k')
    plt.ylim(-0.05,1.05)
    plt.show()

# START analysis

In [9]:
model_name = 'all'
model_type = 'multipl'

# Read in data

In [52]:
confidence_scores = pd.read_csv(f'results/confidence/receptor_rocauc_mean_confidence_scores.csv', index_col = 0)
a_scored_receptors = list(confidence_scores[confidence_scores['Confidence'] == 'A']['Receptor'])
b_scored_receptors = list(confidence_scores[confidence_scores['Confidence'] == 'B']['Receptor'])
c_scored_receptors = list(confidence_scores[confidence_scores['Confidence'] == 'C']['Receptor'])
d_scored_receptors = list(confidence_scores[confidence_scores['Confidence'] == 'D']['Receptor'])


In [10]:
# read in lincs data - design matrix and signatures

In [11]:
lincs_consensus = pd.read_csv(f'data/lincs_consensus/high_quality/lm_all_pert_cell_liana.csv', index_col=0)
lincs_design_matrix = pd.read_csv(f'data/design_matrices/high_quality/all_pert_binary_liana.csv', index_col = 0)

##### CytoSig

In [12]:
# data
cytosig_data = pd.read_csv(f'data/cytosig/diff.merge.gz', compression='gzip', sep = '\t')
cytosig_data = cytosig_data.T
cytosig_data = cytosig_data.fillna(0)

In [13]:
# LINCS data
lincs_data = pd.read_csv(f'data/lincs_consensus/high_quality/lm_{model_name}_pert_cell_liana.csv', index_col = 0)


In [14]:
# LINCS model
lincs_model = pd.read_csv(f'data/coefficient_matrix/high_quality/{model_name}_pert_coef_liana.csv', index_col =0)
# coeff_train = pd.read_csv(f'data//coefficient_matrix/high_quality/train_totalrandomsplit_pert_coef_liana.csv', index_col = 0)
lincs_model = lincs_model.T

In [62]:
lincs_model_A = lincs_model.loc[:, a_scored_receptors]
lincs_model_B = lincs_model.loc[:, b_scored_receptors]
lincs_model_C = lincs_model.loc[:, c_scored_receptors]
lincs_model_D = lincs_model.loc[:, d_scored_receptors]

In [15]:
lr_associations = pd.read_csv(LIG_REC_DF, index_col = 0)
lr_associations = lr_associations[['source_genesymbol', 'target_genesymbol']]

In [16]:
compound_info = pd.read_csv('data/filtered_lincs_meta/filtered_coumpound_info_to_receptor_perturbation_signatures_signed.csv', index_col =0)
compound_info = compound_info[['cmap_name', 'target', 'sign']].reset_index(drop=True)

## Data preparation

In [17]:
# Create perturbation metadata series
lincs_compounds_list = list(compound_info.cmap_name.unique())
lincs_ligand_list = list(lincs_data.index[(lincs_data.reset_index()['index'].str.split('_', expand = True)[2] == 'lig')].str.split('_', expand = True).get_level_values(0).unique())

In [18]:
pert_type_lists = {}
pert_type_lists['compound_list'] = lincs_compounds_list
pert_type_lists['ligand_list'] = list(lr_associations.source_genesymbol) + lincs_ligand_list
pert_type_lists['receptor_list'] = list(lr_associations.target_genesymbol)


# Matching data
- translating from ligand to receptor and vica versa

In [55]:
additional_mapping = {
       'IFNA':'IFNA1', 
       'IFNB':'IFNB1',
       'IFNL':'IFNL1', 
       'TNFA':'TNF', 
       'IL12':'IL12B', 
       'CD40L':'CD40LG', 
       'GMCSF':'CSF2', 
       'MCSF':'CSF1',
       'IL1':'IL1A', 
       'TWEAK':'TNFRSF12A',
       'TRAIL':'TNFSF10',
       'GCSF':'CSF3',
       'IL32':'IL32',
       'OPGL':'TNFSF11',
       'BAFF':'TNFSF13B',
       '41BBL':'TNFSF9',
       'IL23':'IL23A', 
       'CXCL4':'PF4',
       'Activin A':'INHBA',
       'ANGPTL4':'ANGPTL4',
       'ANGPTL2':'ANGPTL2',
       'PGE2':'PTGES2',
       'NO':'NO',
       'PDL1':'CD274'
}
inverse_additional_mapping = {v: k for k, v in additional_mapping.items()}

In [56]:
# lincs
# tmp = lincs_data.index.str.split('_',expand = True).get_level_values(0)
# lincs_pert_types = determine_type_of_perturbation_of_samples(lincs_data.index, tmp, pert_type_lists)
# cytosig
cytosig_pert_types = pd.DataFrame(columns = ['perturbation', 'perturbation_type'], index = cytosig_data.index)
cytosig_pert_types['perturbation_type'] = 'lig'
cytosig_pert_types['perturbation'] = cytosig_pert_types.index.str.split('@', expand = True).get_level_values(0).str.split('&', expand = True).get_level_values(0)
cytosig_pert_types['signed_interactions_lig'] = cytosig_pert_types.apply(lambda x: {x['perturbation']:1}, axis =1)
cytosig_pert_types['perturbation_gs'] = cytosig_pert_types.perturbation.map(lambda x: additional_mapping[x] if x in additional_mapping else x)
cytosig_pert_types['signed_interactions_lig_gs'] = cytosig_pert_types.apply(lambda x: {x['perturbation_gs']:1}, axis =1)

In [57]:
def get_target_list_cp(sample: pd.Series):
    drug = sample.perturbation
    target_dict = compound_info[compound_info['cmap_name'] == drug][['target', 'sign']].set_index('target')['sign'].to_dict()
    return target_dict

def get_target_list_lig(sample: pd.Series):

    receptors = list(lr_associations[lr_associations.source_genesymbol == sample.perturbation]['target_genesymbol'])
    if len(receptors) == 0:
        return np.nan
    
    # sample name PERT_CELL_PERTTYPE (eg. ACVR1_MCF7_oe)
    if (sample.name.split('_')[2] == 'oe'):
        target_dict = dict(zip(receptors, [1]*len(receptors)))
        return target_dict

    # sample name PERT_CELL_PERTTYPE (eg. ACVR1_MCF7_xpr)
    if (sample.name.split('_')[2] == 'xpr'):
        target_dict = dict(zip(receptors, [-1]*len(receptors)))
        return target_dict
    
    # sample name PERT_CELL_PERTTYPE (eg. ACVR1_MCF7_sh)
    if (sample.name.split('_')[2] == 'sh'):
        target_dict = dict(zip(receptors, [-1]*len(receptors)))
        return target_dict

    return np.nan

def get_target_list_rec(sample: pd.Series):
    # if receptor perturbation and overexpression add receptor to target dictionary with sign 1 
    # sample name PERT_CELL_PERTTYPE (eg. RPS19_MCF7_oe)
    if (sample.name.split('_')[2] == 'oe'):
        target_dict = {sample.perturbation:1}
        return target_dict

    # sample name PERT_CELL_PERTTYPE (eg. RPS19_MCF7_xpr)
    if (sample.name.split('_')[2] == 'xpr'):
        target_dict = {sample.perturbation:-1}
        return target_dict
    
    # sample name PERT_CELL_PERTTYPE (eg. RPS19_MCF7_sh)
    if (sample.name.split('_')[2] == 'sh'):
        target_dict = {sample.perturbation:-1}
        return target_dict

    return np.nan
    
def add_target_to_compound_lincs(sample):
    # if compoound perturbation add targets to target dictionary
    if sample.perturbation_type == 'cp':
       return get_target_list_cp(sample)

     # if ligand perturbation - fill return with receptor targets of ligands with sign (use lr_associations (LIANA))
    if sample.perturbation_type == 'lig':
        return get_target_list_lig(sample)
            
    if sample.perturbation_type == 'rec':
        return get_target_list_rec(sample)

    return np.nan
   

In [21]:
# lincs_pert_types['signed_interactions_rec'] = lincs_pert_types.apply(lambda x: add_target_to_compound_lincs(x), axis = 1)


In [58]:
def filter_nan(dict):
  return {key: value for key, value in dict.items() if pd.notna(key) and pd.notna(value)}

In [59]:
def fill_signed_interactions_receptor(lig, sign):
    receptors = list(lr_associations[lr_associations['source_genesymbol'] == lig].target_genesymbol)
    if len(receptors) > 0:
        receptors = dict(zip(receptors, [sign]*len(receptors)))
        return receptors
    else:
        return {np.nan: np.nan}

In [60]:
def translate_ligand_to_receptor(sample_row):
    receptor_dict = dict(map(lambda sample: (sample[0], fill_signed_interactions_receptor(sample[0], sample[1])), sample_row.items()))
    flat_receptor_dict = reduce(lambda all, current: {**all, **filter_nan(current)}, receptor_dict.values(), {})
    if flat_receptor_dict == {}:
        return np.nan
    return flat_receptor_dict

In [61]:
# def fill_signed_interactions_ligand(rec, sign):
#     ligands = list(lr_associations[lr_associations['target_genesymbol'] == rec].source_genesymbol)
#     if len(ligands) > 0:
#         ligands = dict(zip(ligands, [sign]*len(ligands)))
#         return ligands
#     else:
#         return {np.nan: np.nan}

In [26]:
# def translate_receptor_to_ligand(sample_row):
#     ligand_dict = dict(map(lambda sample: (sample[0], fill_signed_interactions_ligand(sample[0], sample[1])), sample_row.items()))
#     flat_ligand_dict = reduce(lambda all, current: {**all, **filter_nan(current)}, ligand_dict.values(), {})
#     if flat_ligand_dict == {}:
#         return np.nan
#     return flat_ligand_dict

In [89]:
cytosig_pert_types['signed_interactions_rec'] = cytosig_pert_types.apply(\
    lambda sample:translate_ligand_to_receptor(sample.signed_interactions_lig_gs), axis = 1)

In [28]:
# lincs_pert_types['signed_interactions_lig'] = lincs_pert_types.apply(\
#     lambda sample: translate_receptor_to_ligand(sample.signed_interactions_rec)\
#         if sample.perturbation_type!='lig' else {sample.perturbation:1}, axis = 1)

In [90]:
metadata = {}
# metadata['lincs'] = lincs_pert_types
metadata['cytosig'] = cytosig_pert_types

# Prediction

# LINCS

In [63]:
# results = {}
# lincs_A = {}
# lincs_B = {}
# lincs_C = {}
# lincs_D = {}
# lincs_all = {}

In [92]:
# lincs_A_gs = {}
# lincs_B_gs = {}
# lincs_C_gs = {}
# lincs_D_gs = {}
# lincs_all_gs = {}

In [64]:
cytokines_A = lr_associations[lr_associations['target_genesymbol'].isin(a_scored_receptors)]['source_genesymbol'].unique()
cytokines_B = lr_associations[lr_associations['target_genesymbol'].isin(b_scored_receptors)]['source_genesymbol'].unique()
cytokines_C = lr_associations[lr_associations['target_genesymbol'].isin(c_scored_receptors)]['source_genesymbol'].unique()
cytokines_D = lr_associations[lr_associations['target_genesymbol'].isin(d_scored_receptors)]['source_genesymbol'].unique()

In [110]:
cytokine_signatures_A = metadata['cytosig'][metadata['cytosig']['perturbation_gs'].isin(cytokines_A)].index
cytokine_signatures_B = metadata['cytosig'][metadata['cytosig']['perturbation_gs'].isin(cytokines_B)].index
cytokine_signatures_C = metadata['cytosig'][metadata['cytosig']['perturbation_gs'].isin(cytokines_C)].index
cytokine_signatures_D = metadata['cytosig'][metadata['cytosig']['perturbation_gs'].isin(cytokines_D)].index


In [93]:
lincs_A_gs['cytosig'] = estimation_with_zscore_calculation_in_chunks(cytosig_data.loc[cytokine_signatures_A], lincs_model_A, number_of_permutation=1000, chunk_size=300)
lincs_B_gs['cytosig'] = estimation_with_zscore_calculation_in_chunks(cytosig_data.loc[cytokine_signatures_B], lincs_model_B, number_of_permutation=1000, chunk_size=300)
lincs_C_gs['cytosig'] = estimation_with_zscore_calculation_in_chunks(cytosig_data.loc[cytokine_signatures_C], lincs_model_C, number_of_permutation=1000, chunk_size=300)
lincs_D_gs['cytosig'] = estimation_with_zscore_calculation_in_chunks(cytosig_data.loc[cytokine_signatures_D], lincs_model_D, number_of_permutation=1000, chunk_size=300)
lincs_all_gs['cytosig'] = estimation_with_zscore_calculation_in_chunks(cytosig_data, lincs_model, number_of_permutation=1000, chunk_size=300)


Number of samples: 894
Number of chunks: 3
Number of permutations: 1000


100%|██████████| 3/3 [01:11<00:00, 23.83s/it]


Number of samples: 935
Number of chunks: 4
Number of permutations: 1000


100%|██████████| 4/4 [01:11<00:00, 17.84s/it]


Number of samples: 817
Number of chunks: 3
Number of permutations: 1000


100%|██████████| 3/3 [00:57<00:00, 19.05s/it]


Number of samples: 831
Number of chunks: 3
Number of permutations: 1000


100%|██████████| 3/3 [00:52<00:00, 17.51s/it]


Number of samples: 2056
Number of chunks: 7
Number of permutations: 1000


100%|██████████| 7/7 [02:19<00:00, 20.00s/it]


In [67]:
lincs_A['cytosig'] = estimation_with_zscore_calculation_in_chunks(cytosig_data.loc[cytokine_signatures_A], lincs_model_A, number_of_permutation=1000, chunk_size=300)
lincs_B['cytosig'] = estimation_with_zscore_calculation_in_chunks(cytosig_data.loc[cytokine_signatures_B], lincs_model_B, number_of_permutation=1000, chunk_size=300)
lincs_C['cytosig'] = estimation_with_zscore_calculation_in_chunks(cytosig_data.loc[cytokine_signatures_C], lincs_model_C, number_of_permutation=1000, chunk_size=300)
lincs_D['cytosig'] = estimation_with_zscore_calculation_in_chunks(cytosig_data.loc[cytokine_signatures_D], lincs_model_D, number_of_permutation=1000, chunk_size=300)
lincs_all['cytosig'] = estimation_with_zscore_calculation_in_chunks(cytosig_data, lincs_model, number_of_permutation=1000, chunk_size=300)


Number of samples: 377
Number of chunks: 2
Number of permutations: 1000


100%|██████████| 2/2 [00:26<00:00, 13.34s/it]


Number of samples: 649
Number of chunks: 3
Number of permutations: 1000


100%|██████████| 3/3 [00:41<00:00, 13.75s/it]


Number of samples: 789
Number of chunks: 3
Number of permutations: 1000


100%|██████████| 3/3 [00:45<00:00, 15.30s/it]


Number of samples: 795
Number of chunks: 3
Number of permutations: 1000


100%|██████████| 3/3 [00:46<00:00, 15.36s/it]


Number of samples: 2056
Number of chunks: 7
Number of permutations: 1000


100%|██████████| 7/7 [02:04<00:00, 17.75s/it]


In [32]:
# if model_type == 'multipl':
    # lincs['cytosig'] = estimation_with_zscore_calculation_in_chunks(cytosig_data, lincs_model, number_of_permutation=1000, chunk_size=300)
    # lincs['lincs'] = estimation_with_zscore_calculation_in_chunks(test_consensus, coeff_train, number_of_permutation=1000, chunk_size=300)

Number of samples: 2056
Number of chunks: 7
Number of permutations: 1000


100%|██████████| 7/7 [02:08<00:00, 18.35s/it]


In [33]:
# lincs['cytosig'] = pd.read_csv('data/cytosig/lincs_predict_cytosig.csv', index_col = 0)

In [68]:
results['lincs_A'] = lincs_A
results['lincs_B'] = lincs_B
results['lincs_C'] = lincs_C
results['lincs_D'] = lincs_D
results['lincs_all'] = lincs_all

In [94]:
results['lincs_A_gs'] = lincs_A_gs
results['lincs_B_gs'] = lincs_B_gs
results['lincs_C_gs'] = lincs_C_gs
results['lincs_D_gs'] = lincs_D_gs
results['lincs_all_gs'] = lincs_all_gs

# CytoSig

In [ ]:
# results['cytosig'] = cytosig

# Create true matrices for ROCAUC analysis

In [35]:
def fill_tm(sample_row, perturbation_info, feature_pert_type):
    sample_row  = pd.Series(index = sample_row.index, name = sample_row.name, dtype = 'int')
    sample_id = sample_row.name

    perturbation_dict = perturbation_info.loc[sample_id, str('signed_interactions_'+feature_pert_type)]
    if pd.notna(perturbation_dict):
        for pert in perturbation_dict.keys():
            if pert in sample_row.index:
                sample_row.loc[pert] = perturbation_dict[pert]
    sample_row = sample_row.fillna(0)
    return sample_row

In [70]:
def create_true_matrices(prediction_results:dict, model:str, feature_pert_type:str, metadata:dict, signature_filter:list):
    print('Create true matrix for predictions of model: ', model)
    truem = {}
    for prediction in prediction_results[model]:
        print(prediction, end = ', ')
        perturbation_info = metadata[prediction].loc[signature_filter]
        data = prediction_results[model][prediction].copy()
        truem[prediction] = data.apply(lambda sample: fill_tm(sample, perturbation_info, feature_pert_type), axis = 1)
    print('done')
    return truem
    

In [71]:
# true_matrices = {}
# true_matrices['lincs'] = create_true_matrices(prediction_results = results, model = 'lincs',feature_pert_type = 'rec', metadata = metadata)

In [108]:
true_matrices = {}
true_matrices['lincs_A'] = create_true_matrices(prediction_results = results, model = 'lincs_A',feature_pert_type = 'rec', metadata = metadata, signature_filter = list(cytokine_signatures_A))
true_matrices['lincs_B'] = create_true_matrices(prediction_results = results, model = 'lincs_B',feature_pert_type = 'rec', metadata = metadata, signature_filter = list(cytokine_signatures_B))
true_matrices['lincs_C'] = create_true_matrices(prediction_results = results, model = 'lincs_C',feature_pert_type = 'rec', metadata = metadata, signature_filter = list(cytokine_signatures_C))
true_matrices['lincs_D'] = create_true_matrices(prediction_results = results, model = 'lincs_D',feature_pert_type = 'rec', metadata = metadata, signature_filter = list(cytokine_signatures_D))
true_matrices['lincs_all'] = create_true_matrices(prediction_results = results, model = 'lincs_all',feature_pert_type = 'rec', metadata = metadata, signature_filter = list(cytosig_data.index))


Create true matrix for predictions of model:  lincs_A
cytosig, done
Create true matrix for predictions of model:  lincs_B
cytosig, done
Create true matrix for predictions of model:  lincs_C
cytosig, done
Create true matrix for predictions of model:  lincs_D
cytosig, done
Create true matrix for predictions of model:  lincs_all
cytosig, done


In [111]:
true_matrices['lincs_A_gs'] = create_true_matrices(prediction_results = results, model = 'lincs_A_gs',feature_pert_type = 'rec', metadata = metadata, signature_filter = list(cytokine_signatures_A))
true_matrices['lincs_B_gs'] = create_true_matrices(prediction_results = results, model = 'lincs_B_gs',feature_pert_type = 'rec', metadata = metadata, signature_filter = list(cytokine_signatures_B))
true_matrices['lincs_C_gs'] = create_true_matrices(prediction_results = results, model = 'lincs_C_gs',feature_pert_type = 'rec', metadata = metadata, signature_filter = list(cytokine_signatures_C))
true_matrices['lincs_D_gs'] = create_true_matrices(prediction_results = results, model = 'lincs_D_gs',feature_pert_type = 'rec', metadata = metadata, signature_filter = list(cytokine_signatures_D))
true_matrices['lincs_all_gs'] = create_true_matrices(prediction_results = results, model = 'lincs_all_gs',feature_pert_type = 'rec', metadata = metadata, signature_filter = list(cytosig_data.index))


Create true matrix for predictions of model:  lincs_A_gs
cytosig, done
Create true matrix for predictions of model:  lincs_B_gs
cytosig, done
Create true matrix for predictions of model:  lincs_C_gs
cytosig, done
Create true matrix for predictions of model:  lincs_D_gs
cytosig, done
Create true matrix for predictions of model:  lincs_all_gs
cytosig, done


In [73]:
# results['cytosig']['lincs'] = results['cytosig']['lincs'].loc[metadata['lincs'].index]

In [74]:
# true_matrices['cytosig'] = create_true_matrices(prediction_results = results, model = 'cytosig',feature_pert_type = 'lig', metadata = metadata)

# Evaluation

In [75]:
def dropna_and_match(base, match):
    base2 = base.dropna(how = 'all')
    match2 = match.loc[base2.index]
    return base2, match2


In [76]:
def calculate_roc_auc(estimated_activities, true_matrix, printoutput= False):

    estimated_activities_matched, true_matrix_matched = dropna_and_match(estimated_activities, true_matrix)
    
    rocaucs = {}
    for ligand in estimated_activities_matched.columns:
        if true_matrix_matched.loc[:, ligand].sum() < 1:
            if printoutput: print(ligand, ' not considered')
            continue
        rocaucs[ligand] = roc_auc_score(y_true = true_matrix_matched.loc[:, ligand], y_score = estimated_activities_matched.loc[:, ligand])
    return rocaucs

In [77]:
def evaluate_with_signature_sign_change_if_inhibitory(activities, binary, printoutput = False, metric = 'ROC'):

    estimated_activities_matched, true_matrix_matched = dropna_and_match(activities, binary)

    print('Evaluation - calculate', metric, 'curves')
    receptors = list(set(activities.columns) & set(binary.columns))
    binary.index  = binary.index.astype('str')
    filtered_activities  = activities.loc[binary.index, receptors]
    filtered_binary = binary.loc[filtered_activities.index,receptors]
    

    metric_results = dict()
    for receptor in filtered_binary.columns:
        sign_changed_activities = filtered_activities.copy()
        sign_changed_binary = filtered_binary.copy()
    
        inhibitory_samples = filtered_binary.loc[(filtered_binary.loc[:, receptor] == -1).values].index
        sign_changed_activities.loc[inhibitory_samples] = sign_changed_activities.loc[inhibitory_samples] * (-1)
        sign_changed_binary.loc[inhibitory_samples, receptor] = 1
        # filter out receptors that are not perturbed in any of the experiments
        if sign_changed_binary.loc[:, receptor].sum() == 0:
            continue
        if metric == 'ROC':
            # rate1: fpr, rate2: tpr
            fpr, tpr, _ = roc_curve(y_true = sign_changed_binary.loc[:, receptor], y_score = sign_changed_activities.loc[:, receptor])
            metric_results[receptor] = auc(fpr, tpr)

        elif metric == 'PR':
            # rate1: precision, rate2: recall
            prec, recall, _ = precision_recall_curve(y_true = sign_changed_binary.loc[:, receptor], probas_pred = sign_changed_activities.loc[:, receptor])
            metric_results[receptor] = auc(prec, recall)

        else:
            raise "No such metric."

    return metric_results

In [112]:
rocaucs = {}
for model in results:
    print('--> Model:', model)
    dataset_rocaucs = {}
    for dataset in results[model]:
        print('Dataset:', dataset, end = '\t')
        dataset_rocaucs[dataset] = evaluate_with_signature_sign_change_if_inhibitory(results[model][dataset], true_matrices[model][dataset], metric = 'ROC', printoutput = False)

    rocaucs[model] = dataset_rocaucs
print('Done')

--> Model: lincs_A
Dataset: cytosig	Evaluation - calculate ROC curves
--> Model: lincs_B
Dataset: cytosig	Evaluation - calculate ROC curves
--> Model: lincs_C
Dataset: cytosig	Evaluation - calculate ROC curves
--> Model: lincs_D
Dataset: cytosig	Evaluation - calculate ROC curves
--> Model: lincs_all
Dataset: cytosig	Evaluation - calculate ROC curves
--> Model: lincs_A_gs
Dataset: cytosig	Evaluation - calculate ROC curves
--> Model: lincs_B_gs
Dataset: cytosig	Evaluation - calculate ROC curves
--> Model: lincs_C_gs
Dataset: cytosig	Evaluation - calculate ROC curves
--> Model: lincs_D_gs
Dataset: cytosig	Evaluation - calculate ROC curves
--> Model: lincs_all_gs
Dataset: cytosig	Evaluation - calculate ROC curves
Done


# Plot

In [113]:
def create_table_to_plot(rocaucs):
    res_df = pd.DataFrame(columns= ['model', 'dataset', 'auc'])
    for model in rocaucs:
        for dataset in rocaucs[model]:
            dataset_rocaucs = pd.DataFrame(columns= res_df.columns)
            dataset_rocaucs.auc = list(rocaucs[model][dataset].values())
            dataset_rocaucs.model = model
            dataset_rocaucs.dataset = dataset
            res_df = pd.concat([res_df, dataset_rocaucs])
    return res_df

In [114]:
def add_median_labels(ax, fmt='.2f'):
    lines = ax.get_lines()
    boxes = [c for c in ax.get_children() if type(c).__name__ == 'PathPatch']
    lines_per_box = int(len(lines) / len(boxes))
    for median in lines[4:len(lines):lines_per_box]:
        x, y = (data.mean() for data in median.get_data())
        # choose value depending on horizontal or vertical plot orientation
        value = x if (median.get_xdata()[1] - median.get_xdata()[0]) == 0 else y
        text = ax.text(x, y, f'{value:{fmt}}', ha='center', va='center',
                       fontweight='bold', color='white')
        # create median-colored border around white text for contrast
        text.set_path_effects([
            path_effects.Stroke(linewidth=3, foreground=median.get_color()),
            path_effects.Normal(),
        ])

In [119]:
def plot_results(results:pd.DataFrame, evaluation_metric = 'ROCAUC', save = False, filename = ''):
    fig, ax = plt.subplots(figsize = (5, 5), facecolor='white')
    boxp = sns.boxplot(data = results, x = 'model', y = 'auc', hue = 'dataset', palette = 'Blues_r') # hue = 'dataset'
    scp = sns.stripplot(data = results, x = 'model', y = 'auc', hue = 'dataset', color = 'k', alpha = 0.3, dodge = True) # hue = 'dataset
    add_median_labels(boxp)
    plt.axhline(0.5, color= 'k')
    # plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    plt.ylabel(evaluation_metric, fontsize = 16)
    plt.xlabel('Model', fontsize = 16)
    plt.xticks(rotation = 90)
    ax.tick_params(axis='both', which='major', labelsize=14)
    # show only bixplot legend
    handles, labels = ax.get_legend_handles_labels()
    length = len(results.dataset.unique())
    l = plt.legend(handles[0:length], labels[0:length], loc = 'upper left', title = 'dataset')
    if save:
        if filename=='': raise "No filename was given."
        # plt.savefig(f'{FIGURES_DIR}model_summary/cytosig/{filename}.png', dpi = 200, bbox_inches = 'tight', transparent = False)
        plt.savefig(f'{FIGURES_DIR}/model_summary/cytosig/{filename}.pdf', bbox_inches = 'tight', transparent = True)


In [116]:
rocaucs_dataframe = create_table_to_plot(rocaucs)

In [83]:
# rocaucs_dataframe.to_csv(f'results/benchmark/cytosig_lincs_rocauc.csv')

In [117]:
table = rocaucs_dataframe.groupby(by=['model', 'dataset']).median().sort_values(by='auc', ascending = False)
table

,,auc
model,dataset,
lincs_B,cytosig,0.664207
lincs_C,cytosig,0.638325
lincs_C_gs,cytosig,0.605521
lincs_all,cytosig,0.600187
lincs_all_gs,cytosig,0.600187
lincs_B_gs,cytosig,0.596264
lincs_D_gs,cytosig,0.591111
lincs_D,cytosig,0.590199
lincs_A_gs,cytosig,0.471006


In [84]:
table = rocaucs_dataframe.groupby(by=['model', 'dataset']).median().sort_values(by='auc', ascending = False)
table

,,auc
model,dataset,
lincs_B,cytosig,0.664207
lincs_C,cytosig,0.638325
lincs_all,cytosig,0.603171
lincs_D,cytosig,0.590199
lincs_A,cytosig,0.442273


In [1]:
plot_results(rocaucs_dataframe, evaluation_metric = 'ROCAUC', save = False, filename = f'cytosig_ROCAUC_comparison_blue')

NameError: name 'plot_results' is not defined